### Installing Packages
We will be installing the rigid-body dynamics library `Pinocchio`, as well as tools to access and visualise robot models,
* The Pinocchio Library

```python
pip install pin
```

Install an example set of robotic systems (containing URDF files and their accompanying visual and geometric files)

```python
pip install example-robot-data
```

Install Meshcat, a visualisation library to display the URDF models

```python
pip install meshcat
```

## Starting Example
Here we will load the manipulator [Panda](https://robodk.com/robot/Franka/Emika-Panda) and load it into the Meshcat visualiser with a desired configuration.

In [ ]:
import pinocchio as pin
import numpy as np
import time

from example_robot_data import load

robot = load('panda')

# Select the default configuration for the robot.
q0 = robot.q0
# Choose a random configuration within the configuration space of the model
# q0 = pin.randomConfiguration(robot.model)

# Display the model
print(robot.model)

## visualise the robot
from pinocchio.visualize import MeshcatVisualizer
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
# Start a new MeshCat server and client.
viz.initViewer(open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
# Display the robot with the initial state q0
viz.display(q0)
# small time window for loading the model 
# if meshcat does not visualise the robot properly, augment the time
# it can be removed in most cases
time.sleep(0.2) 

# Simulating the Dynamics
We will now use the rigid-body dynamics algorithms to forward-compute the dynamics of the robot from an initial state

In [50]:
import pinocchio as pin
import numpy as np
import time

from example_robot_data import load

robot = load('panda')

# Select the default configuration for the robot.
q0 = robot.q0
# Choose a random configuration within the configuration space of the model
# q0 = pin.randomConfiguration(robot.model)

# Display the model
print(robot.model)

## visualise the robot
from pinocchio.visualize import MeshcatVisualizer
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
# Start a new MeshCat server and client.
viz.initViewer(open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
# Display the robot with the initial state q0
viz.display(q0)
# small time window for loading the model 
# if meshcat does not visualise the robot properly, augment the time
# it can be removed in most cases
time.sleep(0.2) 

# Create velocity and acceleration vectors to represent the state of the system
q = np.zeros(robot.nq)
v = np.zeros(robot.nv)
a = np.zeros(robot.nv)

# Control inputs to each joint of the robot (set to zero) 
u = np.zeros(robot.nq)


# Simulation time (s)
T = 3.0

# Resolution (s)
dt = 0.001

# Current time
t = 0.0

# while t <= T:
#     # Calculate the current acceleration of the robot given the configuration, velocity and input
#     a = pin.aba(robot.model, robot.data, q, v, u)
#     # Forward step
#     v = v + dt * a
#     q = q + dt * v
    
#     t += dt
    
#     viz.display(q)
#     time.sleep(dt)

Nb joints = 10 (nq=9,nv=9)
  Joint 0 universe: parent=0
  Joint 1 panda_joint1: parent=0
  Joint 2 panda_joint2: parent=1
  Joint 3 panda_joint3: parent=2
  Joint 4 panda_joint4: parent=3
  Joint 5 panda_joint5: parent=4
  Joint 6 panda_joint6: parent=5
  Joint 7 panda_joint7: parent=6
  Joint 8 panda_finger_joint1: parent=7
  Joint 9 panda_finger_joint2: parent=7

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7017/static/


## Adding a Controller
Let us design a PD controller to control the arm to its nominal pose from a random configuration

In [54]:
# Control Law u = Kp (q - qr) + Kd (v - vr)
qr = robot.q0
vr = np.zeros(robot.nv)

Kp = np.diag([20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0])
Kd = np.diag([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

t = 0.0
frame_cnt = 0
while t <= T:
    # Compute control
    u = Kp.dot(qr - q) + Kd.dot(vr - v)

    # Calculate the current acceleration of the robot given the configuration, velocity and input
    a = pin.aba(robot.model, robot.data, q, v, u)
    
    # Forward step
    v = v + dt * a
    q = q + dt * v
    
    t += dt
    # Display every 24 frames
    if frame_cnt % 30 == 0:
        viz.display(q)
        time.sleep(0.03)
    
    frame_cnt += 1

## Create a Computed-Torque Controller
Given the difficulty in tuning a direct PD controller, let us include torque compensation from the dynamics

In [55]:
t = 0.0
T = 10.0
frame_cnt = 0
while t <= T:
    # Computed torque method
    # Set desired acceleration of the system as the PD error of the state
    ad = Kp.dot(qr - q) + Kd.dot(vr - v)
    # Generate torque outputs that achieve these accelerations
    u = pin.rnea(robot.model, robot.data, q, v, ad)

    # Calculate the current acceleration of the robot given the configuration, velocity and input
    a = pin.aba(robot.model, robot.data, q, v, u)
    
    # Forward step
    v = v + dt * a
    q = q + dt * v
    
    t += dt
    # Display a frame roughly every 33 ms
    if frame_cnt % 33 == 0:
        viz.display(q)
        time.sleep(0.033)
    
    frame_cnt += 1